
# Multi-Agent Orchestration with LangGraph

This notebook demonstrates a **Planner → Worker → Critic** multi-agent pattern using LangGraph.

We simulate:
- A **Planner Agent** that decomposes a high-level request
- A **Worker Agent** that writes the answer
- A **Critic Agent** that reviews quality and requests fixes



## 1. Setup


In [ ]:

%pip install -q langgraph langchain-openai langchain

import os
os.environ.setdefault("OPENAI_API_KEY", "sk-REPLACE_ME")



## 2. Define the Agents (Planner, Worker, Critic)


In [ ]:

from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

def planner_node(state: MessagesState):
    user_msg = state["messages"][0]["content"]
    prompt = (
        "You are a planning agent. Break the following request into 3-5 concrete steps "
        "for a worker agent. Return steps in a numbered list.\n\n"
        f"Request: {user_msg}"
    )
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": state["messages"] + [{"role": "assistant", "name": "planner", "content": resp.content}]}

def worker_node(state: MessagesState):
    user_msg = state["messages"][0]["content"]
    plan = state["messages"][-1]["content"]
    prompt = (
        "You are a worker agent. Follow the plan below to produce a high-quality answer.\n\n"
        f"User request: {user_msg}\n\n"
        f"Plan: {plan}"
    )
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": state["messages"] + [{"role": "assistant", "name": "worker", "content": resp.content}]}

def critic_node(state: MessagesState):
    draft = state["messages"][-1]["content"]
    prompt = (
        "You are a critic agent. Review the draft answer for clarity, completeness, and correctness.\n"
        "If it is good, respond with 'ACCEPT' followed by a short justification.\n"
        "If it needs improvement, respond with 'REVISE' and give concrete instructions.\n\n"
        f"Draft answer:\n{draft}"
    )
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": state["messages"] + [{"role": "assistant", "name": "critic", "content": resp.content}]}



### Routing Node: Decide Whether to Revise

If Critic says **REVISE**, we send the instructions back to Worker.  
If Critic says **ACCEPT**, we end.


In [ ]:

def router_node(state: MessagesState):
    critic_msg = state["messages"][-1]["content"]
    text = critic_msg if isinstance(critic_msg, str) else str(critic_msg)
    if text.strip().upper().startswith("REVISE"):
        return "worker"
    return "end"



## 3. Build the Multi-Agent Graph


In [ ]:

builder = StateGraph(MessagesState)
builder.add_node("planner", planner_node)
builder.add_node("worker", worker_node)
builder.add_node("critic", critic_node)
builder.add_node("router", router_node)

builder.add_edge(START, "planner")
builder.add_edge("planner", "worker")
builder.add_edge("worker", "critic")
builder.add_edge("critic", "router")

# Conditional edges from router
builder.add_edge("router", "worker", condition=lambda s: router_node(s) == "worker")
builder.add_edge("router", END, condition=lambda s: router_node(s) == "end")

multiagent_graph = builder.compile()
multiagent_graph



## 4. Run a Multi-Agent Orchestration Example

We ask the system to **design a small agentic AI demo**.  
Planner breaks it down, Worker writes, Critic reviews and maybe sends it back.


In [ ]:

query = (
    "Design a simple demo of an agent that uses MCP tools and RAG to answer questions about internal docs. "
    "Explain the components and the flow."
)

state = {"messages": [{"role": "user", "content": query}]}
result = multiagent_graph.invoke(state)

for i, m in enumerate(result["messages"]):
    print(f"\n[{i}] role={m['role']} name={m.get('name')}")
    print(m["content"])



## 5. Takeaways

- You can treat **each agent as a node** in a LangGraph.  
- Coordination logic (like *when to re-invoke Worker*) can be encoded as routing nodes.  
- This pattern generalizes to:
  - Research assistant teams
  - Code review committees
  - Autonomous enterprise workflows  
